<a href="https://colab.research.google.com/github/keval47/Machine-Learning/blob/master/%20Multi-Class%20Feed%20Forward%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [384]:
data = datasets.load_iris()
X = data.data
Y = data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X,Y,random_state = 5)

y_train = y_train.reshape(len(y_train),1)
y_test = y_test.reshape(len(y_test),1)


print(X_train.shape,y_train.shape)

(112, 4) (112, 1)


In [0]:
def OneHotEncoder(x):
    arr = np.zeros((len(x),np.max(x)+1),dtype=int)
    for i,val in enumerate(x):
        arr[i][val[0]] = 1
    return arr

In [386]:
y_train = OneHotEncoder(y_train)
print(X_train.shape,y_train.shape)

(112, 4) (112, 3)


In [0]:
class Neuron:
    def __init__(self,inputNeuron,hiddenNeuron,outputNeuron):
        self.w_ih = np.random.random((inputNeuron,hiddenNeuron)) # (2x3) weights b/w input and hidden layer         
        self.b_ih = np.random.random((1,hiddenNeuron)) #(1x3) bias b/w input and hidden layer

        self.w_ho = np.random.random((hiddenNeuron,outputNeuron)) # (3x1) weights b/w hidden and output layer
        self.b_ho = np.random.random((1,outputNeuron)) #(1x1) bias b/w  hidden and output layer

        self.hiddenLayerOutput = np.array((hiddenNeuron,1),dtype=np.float64) # (3x1) just for store output generated by model
        self.outputLayerOutput = np.array((outputNeuron,1),dtype=np.float64) # (1x1) just for store output generated by model

In [0]:
class ffnn:
    def __init__(self):
        pass

    def layers(self,inputNeuron,hiddenNeuron,outputNeuron):
        self._neuron = Neuron(inputNeuron,hiddenNeuron,outputNeuron)# creating neurons

    def hyper_params(self,lr=0.5,epochs=1000):
        self._lr = lr # learning rate
        self._epochs = epochs #iteration how many times data will be train

    def fit(self,X,Y):
        self._x = X
        self._y = Y

    # function for matrix multiplication you can also use np.dot
    def _dot(self,a,b):
        matmul = []
        for val in a:
            valSum = 0
            for i in range(len(val)):
                valSum += val[i] * b[i]
            matmul.append(valSum)
        return np.array(matmul)
    
    # squzze value in between 1/0. return probability
    def _sigmoid(self,z,deriv=False):
        if(deriv):
            return z * (1-z)
        return 1/(1+(np.exp(-z)))
    
    def _softmax(self,x):

        # unstable
        # x = x-x.max()
        # expA = np.exp(x)         
        # return expA / expA.sum()   

        #stable
        xrel = x - x.max(axis=1, keepdims=True)# make every value 0 or below, as exp(0) won't overflow
        exp_xrel = np.exp(xrel)
        return exp_xrel / exp_xrel.sum(axis=1, keepdims=True) 
    
    # return weighted sum. return output based on weights
    def _feedforward(self):
        self._neuron.hiddenLayerOutput = self._sigmoid(self._dot(self._x,self._neuron.w_ih)+self._neuron.b_ih)
        self._neuron.outputLayerOutput = self._softmax(self._dot(self._neuron.hiddenLayerOutput,self._neuron.w_ho)+self._neuron.b_ho)

    # perform backpropagation. adjusts weights that let us close to accurate output 
    def _backProp(self):
        # print(self._sigmoid(self._neuron.outputLayerOutput))
        # print("\n")
        # print(self._sigmoid(self._neuron.outputLayerOutput,True))
        # return
        err_ho = self._y - self._neuron.outputLayerOutput #calculate gradient of err. gradient=direction
        slope_ho = self._sigmoid(self._neuron.outputLayerOutput,True) # calculate slope of output
        delta_ho = err_ho * slope_ho #change in w_ho weights. define direction that make less error

        err_ih = self._dot(delta_ho,self._neuron.w_ho.T)  # define gradient of err based on output layer error
        slope_ih = self._sigmoid(self._neuron.hiddenLayerOutput,True) #calculate slope of hidden layer output
        delta_ih = err_ih * slope_ih #change in i_ho weights. define direction that make less error
        
        # adjust weights to make less err
        self._neuron.w_ih += self._dot(self._x.T,delta_ih) * self._lr #step further to defined direction by delta.step = learning rate
        self._neuron.w_ho += self._dot(self._neuron.hiddenLayerOutput.T,delta_ho) * self._lr #step further to defined direction by delta. step = learning rate

         # adjust bias to make less err
        self._neuron.b_ih += np.sum(delta_ih,axis=0) * self._lr #step further to defined direction by delta. step = learning rate
        self._neuron.b_ho += np.sum(delta_ho,axis=0) * self._lr #step further to defined direction by delta. step = learning rate

    def _err(self):
        # mean squared err
        err = np.mean(np.square(self._y - self._neuron.outputLayerOutput),dtype=np.float64)
        return err

    def predict(self,val):
        # predict probability
        # if you want to predict class then simply put threshold and filtered out class
        ho = self._sigmoid(self._dot(val,self._neuron.w_ih)+self._neuron.b_ih)
        return self._softmax(self._dot(ho,self._neuron.w_ho)+self._neuron.b_ho)
        # return self._sigmoid(self._dot(val,self._neuron.w_ho)+self._neuron.b_ho)

    def train(self,printErr=False):
        # itrating over and over to make model train
        for i in range(self._epochs):
            self._feedforward()
            self._backProp()
            if(printErr):
                # print err by every 100 iteration
                if(i % 100 == 0):
                    print(self._err())
        return "training finished"

In [0]:
# running model
neuralnet = ffnn()
neuralnet.hyper_params(lr=0.6,epochs=1000)
neuralnet.layers(4,4,3)
neuralnet.fit(X_train,y_train)
neuralnet.train(printErr=True)

In [0]:
predicts = np.zeros((len(X_test),1),dtype=int)
for x in range(len(X_test)):
    p =  neuralnet.predict([X_test[x]])
    predicts[x] = np.argmax(p)

In [391]:
accuracy_score(y_test,predicts)

0.9473684210526315